In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [3]:
customers.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [4]:
products.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [5]:
transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [6]:
result = pd.merge(left=pd.merge(left=transactions,right=customers,on='CustomerID',how='inner'),right=products,on='ProductID',how='inner')

In [7]:
result.drop(columns='Price_x',axis=1,inplace=True)
result.rename(columns={'Price_y':'Price'},inplace=True)

In [8]:
ordered_result = result[['CustomerID','CustomerName','Region','SignupDate','ProductID','ProductName','Category','TransactionDate','Price','Quantity','TotalValue']]

In [9]:
ordered_result.head()

,CustomerID,CustomerName,Region,SignupDate,ProductID,ProductName,Category,TransactionDate,Price,Quantity,TotalValue
0,C0199,Andrea Jenkins,Europe,2022-12-03,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-08-25 12:38:23,300.68,1,300.68
1,C0146,Brittany Harvey,Asia,2024-09-04,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-05-27 22:23:54,300.68,1,300.68
2,C0127,Kathryn Stevens,Europe,2024-04-04,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-04-25 07:38:55,300.68,1,300.68
3,C0087,Travis Campbell,South America,2024-04-11,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-03-26 22:55:37,300.68,2,601.36
4,C0070,Timothy Perez,Europe,2022-03-15,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-03-21 15:10:10,300.68,3,902.04


In [10]:
ordered_result.dtypes

CustomerID          object
CustomerName        object
Region              object
SignupDate          object
ProductID           object
ProductName         object
Category            object
TransactionDate     object
Price              float64
Quantity             int64
TotalValue         float64
dtype: object

In [11]:
ordered_result['SignupDate'] = pd.to_datetime(ordered_result['SignupDate'])
ordered_result['TransactionDate'] = pd.to_datetime(ordered_result['TransactionDate'])
ordered_result.dtypes

C:\Users\saswa\AppData\Local\Temp\ipykernel_12232\790954055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_result['SignupDate'] = pd.to_datetime(ordered_result['SignupDate'])
C:\Users\saswa\AppData\Local\Temp\ipykernel_12232\790954055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_result['TransactionDate'] = pd.to_datetime(ordered_result['TransactionDate'])


CustomerID                 object
CustomerName               object
Region                     object
SignupDate         datetime64[ns]
ProductID                  object
ProductName                object
Category                   object
TransactionDate    datetime64[ns]
Price                     float64
Quantity                    int64
TotalValue                float64
dtype: object

In [12]:
ordered_result.head()

,CustomerID,CustomerName,Region,SignupDate,ProductID,ProductName,Category,TransactionDate,Price,Quantity,TotalValue
0,C0199,Andrea Jenkins,Europe,2022-12-03,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-08-25 12:38:23,300.68,1,300.68
1,C0146,Brittany Harvey,Asia,2024-09-04,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-05-27 22:23:54,300.68,1,300.68
2,C0127,Kathryn Stevens,Europe,2024-04-04,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-04-25 07:38:55,300.68,1,300.68
3,C0087,Travis Campbell,South America,2024-04-11,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-03-26 22:55:37,300.68,2,601.36
4,C0070,Timothy Perez,Europe,2022-03-15,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-03-21 15:10:10,300.68,3,902.04


#### >Let's extract "Month","Day","Hour","Minute","Seconds" from TransactionDate column

In [13]:
ordered_result['Month'] = pd.to_datetime(ordered_result['TransactionDate']).dt.month
ordered_result['Day'] = pd.to_datetime(ordered_result['TransactionDate']).dt.day
ordered_result['Hour'] = pd.to_datetime(ordered_result['TransactionDate']).dt.hour
ordered_result['Minute'] = pd.to_datetime(ordered_result['TransactionDate']).dt.minute
ordered_result['Seconds'] = pd.to_datetime(ordered_result['TransactionDate']).dt.second

In [14]:
ordered_result.head()

,CustomerID,CustomerName,Region,SignupDate,ProductID,ProductName,Category,TransactionDate,Price,Quantity,TotalValue,Month,Day,Hour,Minute,Seconds
0,C0199,Andrea Jenkins,Europe,2022-12-03,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-08-25 12:38:23,300.68,1,300.68,8,25,12,38,23
1,C0146,Brittany Harvey,Asia,2024-09-04,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-05-27 22:23:54,300.68,1,300.68,5,27,22,23,54
2,C0127,Kathryn Stevens,Europe,2024-04-04,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-04-25 07:38:55,300.68,1,300.68,4,25,7,38,55
3,C0087,Travis Campbell,South America,2024-04-11,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-03-26 22:55:37,300.68,2,601.36,3,26,22,55,37
4,C0070,Timothy Perez,Europe,2022-03-15,P067,ComfortLiving Bluetooth Speaker,Electronics,2024-03-21 15:10:10,300.68,3,902.04,3,21,15,10,10


#### >Let's perform encoding for selected columns like "Region" and "Category" using get_dummies method

In [15]:
encoded_cols = pd.get_dummies(ordered_result[['Region','Category']])
encoded_cols.head()

,Region_Asia,Region_Europe,Region_North America,Region_South America,Category_Books,Category_Clothing,Category_Electronics,Category_Home Decor
0,0,1,0,0,0,0,1,0
1,1,0,0,0,0,0,1,0
2,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0


##### >Features that we gonna consider further

In [16]:
features = ordered_result[["Price", "Quantity", "TotalValue", "Month","Day","Hour","Minute","Seconds"]]
features.head()

,Price,Quantity,TotalValue,Month,Day,Hour,Minute,Seconds
0,300.68,1,300.68,8,25,12,38,23
1,300.68,1,300.68,5,27,22,23,54
2,300.68,1,300.68,4,25,7,38,55
3,300.68,2,601.36,3,26,22,55,37
4,300.68,3,902.04,3,21,15,10,10


#### >Let's merge the encoded col's and the features that we have selected

In [17]:
final_features = features.join(encoded_cols)
final_features.head()

,Price,Quantity,TotalValue,Month,Day,Hour,Minute,Seconds,Region_Asia,Region_Europe,Region_North America,Region_South America,Category_Books,Category_Clothing,Category_Electronics,Category_Home Decor
0,300.68,1,300.68,8,25,12,38,23,0,1,0,0,0,0,1,0
1,300.68,1,300.68,5,27,22,23,54,1,0,0,0,0,0,1,0
2,300.68,1,300.68,4,25,7,38,55,0,1,0,0,0,0,1,0
3,300.68,2,601.36,3,26,22,55,37,0,0,0,1,0,0,1,0
4,300.68,3,902.04,3,21,15,10,10,0,1,0,0,0,0,1,0


#### > Let's perform scaling to the final_features

In [18]:
scaler = StandardScaler()
normalized_features = scaler.fit_transform(final_features)


In [19]:
normalized_features

array([[ 0.19994831, -1.37548802, -0.7898504 , ..., -0.54344897,
         1.71377008, -0.57427105],
       [ 0.19994831, -1.37548802, -0.7898504 , ..., -0.54344897,
         1.71377008, -0.57427105],
       [ 0.19994831, -1.37548802, -0.7898504 , ..., -0.54344897,
         1.71377008, -0.57427105],
       ...,
       [-1.80380521,  1.30926413, -1.24714597, ...,  1.84009916,
        -0.58350885, -0.57427105],
       [-1.80380521,  0.41434675, -1.28532833, ...,  1.84009916,
        -0.58350885, -0.57427105],
       [-1.80380521, -0.48057063, -1.32351068, ...,  1.84009916,
        -0.58350885, -0.57427105]])

In [20]:
similarity_matrix = cosine_similarity(normalized_features) # Compute similarity matrix

#### > Function to find top 3 similar customers

In [21]:
def find_similar_customers(customer_id, df, similarity_matrix):
    idx_list = df[df["CustomerID"] == customer_id].index.tolist()
    if not idx_list:
        return "Customer not found."
    idx = idx_list[0]
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_customers = [(df.iloc[i]["CustomerID"], df.iloc[i]["CustomerName"], score) 
                          for i, score in similarity_scores[1:4]]
    return similar_customers

In [22]:
print(find_similar_customers("C0199", ordered_result, similarity_matrix))

[('C0051', 'Nicholas Ellis', 0.9796755200474737), ('C0127', 'Kathryn Stevens', 0.9508286324158539), ('C0172', 'Jamie Webb', 0.8847619666259156)]


#### >To find the top 3 lookalikes with there similarity scores for the first 20 customers, we have to sort the DF first

In [39]:
ordered_result = ordered_result.sort_values(by="CustomerID")
ordered_result.head()

,CustomerID,CustomerName,Region,SignupDate,ProductID,ProductName,Category,TransactionDate,Price,Quantity,TotalValue,Month,Day,Hour,Minute,Seconds
791,C0001,Lawrence Carroll,South America,2022-07-10,P054,SoundWave Cookbook,Books,2024-01-19 03:12:55,57.30,2,114.60,1,19,3,12,55
433,C0001,Lawrence Carroll,South America,2022-07-10,P083,ActiveWear Smartwatch,Electronics,2024-05-07 03:11:44,455.72,2,911.44,5,7,3,11,44
21,C0001,Lawrence Carroll,South America,2022-07-10,P022,HomeSense Wall Art,Home Decor,2024-09-17 09:01:18,137.54,3,412.62,9,17,9,1,18
447,C0001,Lawrence Carroll,South America,2022-07-10,P096,SoundWave Headphones,Electronics,2024-04-08 00:01:00,307.47,2,614.94,4,8,0,1,0
84,C0001,Lawrence Carroll,South America,2022-07-10,P029,TechPro Headphones,Electronics,2024-11-02 17:04:16,433.64,3,1300.92,11,2,17,4,16


##### >Made a dictonary to map CustomerID to Lookalikes 

In [42]:
Customer_Lookalikes = {}

for customer_id in ordered_result["CustomerID"].iloc[:95]:
    Customer_Lookalikes[customer_id] = find_similar_customers(customer_id, ordered_result, similarity_matrix)
    
lookalike_df = pd.DataFrame(list(Customer_Lookalikes.items()), columns=["CustomerID", "Lookalikes"])
lookalike_df


,CustomerID,Lookalikes
0,C0001,"[(C0164, Morgan Perez, 0.9520203379183898), (C..."
1,C0002,"[(C0054, Bruce Rhodes, 0.9371551384638416), (C..."
2,C0003,"[(C0056, Erika Fernandez, 0.9345054017183329),..."
3,C0004,"[(C0039, Angela Harris, 0.9504392375331024), (..."
4,C0005,"[(C0177, Julia Kelly, 0.9250769510160148), (C0..."
5,C0006,"[(C0168, Karen Clements MD, 0.9316182611595348..."
6,C0007,"[(C0021, Robert Blanchard, 0.9030262970651577)..."
7,C0008,"[(C0021, Robert Blanchard, 0.8316026261456988)..."
8,C0009,"[(C0099, Rodney Eaton, 0.91238054156253), (C00..."
9,C0010,"[(C0147, Hunter Fuller, 0.8878489709832955), (..."


#### > Converting the DF to .CSV file

In [43]:
lookalike_df.to_csv("Saswat_Mishra_Lookalike.csv",index=False)